# importing the neccessary libraries

In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

In [2]:
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings("ignore")

In [3]:
import tensorflow as tf
from keras.optimizers import *
from keras import regularizers
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import text

Using TensorFlow backend.


In [4]:
import pickle

# reading the data

In [5]:
train=pd.read_csv("E:/TCS_data/train.csv",nrows=150000)
#test=pd.read_csv("E:/TCS_data/Test.csv",nrows=100000)

In [6]:
print(train.columns) # getting the columns

Index(['Id', 'Title', 'Body', 'Tags'], dtype='object')


In [7]:
train.head()
#print(train.columns)

,Id,Title,Body,Tags
0,1,How to check if an uploaded file is an image w...,<p>I'd like to check if an uploaded file is an...,php image-processing file-upload upload mime-t...
1,2,How can I prevent firefox from closing when I ...,"<p>In my favorite editor (vim), I regularly us...",firefox
2,3,R Error Invalid type (list) for variable,<p>I am import matlab file and construct a dat...,r matlab machine-learning
3,4,How do I replace special characters in a URL?,"<p>This is probably very simple, but I simply ...",c# url encoding
4,5,How to modify whois contact details?,<pre><code>function modify(.......)\n{\n $mco...,php api file-get-contents


In [8]:
train.shape

(150000, 4)

# preprocessing the data

In [9]:
#function to remove html tags from data
def remove_html(text):
    tag = re.compile(r'<.*?>')
    return tag.sub('', text)

In [10]:
#removing the html tags from Body column
for i in range(train.shape[0]):
    train.loc[i, 'Body'] = remove_html(train.loc[i,'Body'])

In [11]:
train.head()

,Id,Title,Body,Tags
0,1,How to check if an uploaded file is an image w...,I'd like to check if an uploaded file is an im...,php image-processing file-upload upload mime-t...
1,2,How can I prevent firefox from closing when I ...,"In my favorite editor (vim), I regularly use c...",firefox
2,3,R Error Invalid type (list) for variable,I am import matlab file and construct a data f...,r matlab machine-learning
3,4,How do I replace special characters in a URL?,"This is probably very simple, but I simply can...",c# url encoding
4,5,How to modify whois contact details?,function modify(.......)\n{\n $mcontact = fil...,php api file-get-contents


In [12]:
def clean_words(sentence):#cleaning the text data
    sentence = sentence.lower()                # Converting to lowercase
    rem=re.compile(r'[^\w]')
    sentence=re.sub(rem,r' ', sentence)
    sentence=re.sub(r'(?:^| )\w(?:$| )', r' ', sentence)#removing single characters
    sentence = re.sub(r'[?$|!|\':|"|#]',r' ',sentence)
    sentence = re.sub(r'[.|,|)|(|\|/]\"\"]',r' ',sentence)#Removing Punctuations
    sentence=re.sub(r'\d+',' ',sentence)
    return sentence

In [13]:
#removing  unneccessary symbols from
for column in train:
    if(column=='Title' or column=='Body'):
        for i in range(train.shape[0]):
            train.loc[i, column] =clean_words(train.loc[i,column])
# for column in train:
#     index=0
#     if(column=='Title' or column=='Body'):
#         for sentence in train[column]:
#             cleaned_word=clean_words(sentence)
#             train[column][index]=cleaned_word  
#             index+=1

In [14]:
train.head()

,Id,Title,Body,Tags
0,1,how to check if an uploaded file is an image w...,d like to check if an uploaded file is an ima...,php image-processing file-upload upload mime-t...
1,2,how can prevent firefox from closing when pres...,in my favorite editor vim regularly use ctr...,firefox
2,3,error invalid type list for variable,am import matlab file and construct data fram...,r matlab machine-learning
3,4,how do replace special characters in url,this is probably very simple but simply canno...,c# url encoding
4,5,how to modify whois contact details,function modify mcontact file_...,php api file-get-contents


In [15]:
#stop words 
stop_words=stopwords.words('english')

In [16]:
#tokenization
tokenized_doc_body=train['Body'].apply(lambda x:x.split())
tokenized_doc_title=train['Title'].apply(lambda x:x.split())
#remove stop_words
tokenized_doc_body=tokenized_doc_body.apply(lambda x:[item for item in x if item not in stop_words])
tokenized_doc_title=tokenized_doc_title.apply(lambda x:[item for item in x if item not in stop_words])
#de-tokenization
detokenized_doc_body=[]
detokenized_doc_title=[]
for i in range(train.shape[0]):
    t1=' '.join(tokenized_doc_body[i])
    t2=' '.join(tokenized_doc_title[i])
    detokenized_doc_body.append(t1)
    detokenized_doc_title.append(t2)
train['Body']= detokenized_doc_body
train['Title']= detokenized_doc_title
# def stop_words_remove(text):
#     tokenize_text=text.apply(lambda x:x.split())
#     final=tokenize_text.apply(lambda x:[item for item in x if item not in stop_words])
#     #de-tokenize
#     de_token=' '.join(final)
#     return de_token

In [17]:
train.head(5)
# for column in train:
#     if(column=='Title' or column=='Body'):
#         for i in range(train.shape[0]):
#             train.loc[i, column] =stop_words_remove(train.loc[i,column])

,Id,Title,Body,Tags
0,1,check uploaded file image without mime type,like check uploaded file image file g png jpg ...,php image-processing file-upload upload mime-t...
1,2,prevent firefox closing press ctrl,favorite editor vim regularly use ctrl execute...,firefox
2,3,error invalid type list variable,import matlab file construct data frame matlab...,r matlab machine-learning
3,4,replace special characters url,probably simple simply cannot find answer basi...,c# url encoding
4,5,modify whois contact details,function modify mcontact file_get_contents htt...,php api file-get-contents


# encoding the tags using multilabelbinarizer

In [18]:
train=shuffle(train,random_state=5)

In [19]:
train.head()

,Id,Title,Body,Tags
76752,76753,dynamic element width ie subpixel precision,divs left center right want position line spec...,javascript jquery css
120848,120849,mail merge vista word windows live mail,mail merge send email word windows live mail i...,microsoft-word microsoft-office office-2007 wi...
131453,131454,cannot receive greeting message remote mail se...,trying setup mail server postfix amp dovecot u...,linux ubuntu email postfix mailserver
67124,67125,txt separate strings,trying take chars txt file words text become s...,c string text-files strtok
129100,129101,changing flash files screen clicking button,trying develop way change flash file displayed...,flash


In [20]:
#encode tags to multi-hot
train_tags=[]
for tag in train['Tags'].values:
    tags=[i for i in tag.split()]
    train_tags.append(tags)
#print(train_topics,'\n')
topic_encoder=MultiLabelBinarizer()
topic_encoded=topic_encoder.fit_transform(train_tags)

In [21]:
num_tags=len(topic_encoded[0])
print(num_tags)
print(topic_encoder.classes_)
print(topic_encoded[0])

21528
['.a' '.aspxauth' '.bash-profile' ... 'zypper' 'zywall' 'zyxel']
[0 0 0 ... 0 0 0]


# to get document term matrix we use this function

In [22]:
#tokenize
class TextPreprocessor(object):
    def __init__(self,vocab_size):
        self._vocab_size=vocab_size
        self._tokenizer=None
    def create_tokenizer(self,text_list):
        tokenizer=text.Tokenizer(num_words=self._vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
        tokenizer.fit_on_texts(text_list)
        self._tokenizer=tokenizer
    def transform_text(self,text_list):
        text_matrix=self._tokenizer.texts_to_matrix(text_list)
        return text_matrix

In [23]:
train_body=train['Body'].values
#train_title=train['Title'].values
# #test_text=test_data['Review Text'].values
# #test_title=test_data['Review Title'].values
# for i in range(train.shape[0]):
#     train[i]=train_text[i]+train_title[i]
# for i in range(test_text.shape[0]):
#     test_text[i]=test_text[i]+test_title[i]

In [24]:
vocab_size=600
processor=TextPreprocessor(vocab_size)
processor.create_tokenizer(train_body)
body_text=processor.transform_text(train_body)
#body_title=processor.transform_text(train_title)

In [25]:
with open('./processor_state_f1.pkl','wb') as f:
    pickle.dump(processor,f)

# horizontally stacking up the  body and title column term matrices

In [26]:
#total_data=np.hstack(body_tile,body_text)

In [27]:
print(len(body_text[0]))
print(body_text[0])

600
[0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [28]:
print(body_text.shape) #  final size of the term matrix 

(150000, 600)


In [29]:
words=processor._tokenizer.word_index
word_lookup=list()
for i in words.keys():
    word_lookup.append(i)
print(len(word_lookup))

333205


# neural network

In [30]:
#load_processor=pickle.load(open('processor_state_f.pkl','rb'))

In [30]:
#create model
#def create_model(num_topics,vocab_size):
model=Sequential()
model.add(Dense(128,input_shape=(vocab_size,),activation='relu'))
model.add(Dense(50,activation='relu' ))
model.add(Dropout(0.5))
#model.add(Dense(32,activation='relu'))
model.add(Dense(30,activation='relu' ))
model.add(Dropout(0.5))
model.add(Dense(num_tags,activation='sigmoid'))
#adam=adam(lr=0.0001,beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#model=create_model(num_tags,vocab_size)
model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

W0921 18:25:28.270692  3076 deprecation.py:506] From c:\python 3.6\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0921 18:25:28.706672  3076 deprecation.py:323] From c:\python 3.6\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               76928     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                6450      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 30)                1530      
_________________________________________________________________
dropout_1 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 21528)             667368    
Total params: 752,276
Trainable params: 752,276
Non-trainable params: 0
__________________________________________________

In [31]:
model.fit(body_text,topic_encoded,epochs=31,batch_size=128,validation_split=0.1)

Train on 135000 samples, validate on 15000 samples
Epoch 1/31
135000/135000 [==============================] - 206s 2ms/sample - loss: 0.6931 - acc: 0.5061 - val_loss: 0.6930 - val_acc: 0.5199
Epoch 2/31
135000/135000 [==============================] - 213s 2ms/sample - loss: 0.6929 - acc: 0.5189 - val_loss: 0.6928 - val_acc: 0.5413
Epoch 3/31
135000/135000 [==============================] - 210s 2ms/sample - loss: 0.6927 - acc: 0.5316 - val_loss: 0.6926 - val_acc: 0.5622
Epoch 4/31
135000/135000 [==============================] - 188s 1ms/sample - loss: 0.6925 - acc: 0.5440 - val_loss: 0.6925 - val_acc: 0.5826
Epoch 5/31
135000/135000 [==============================] - 202s 1ms/sample - loss: 0.6923 - acc: 0.5562 - val_loss: 0.6923 - val_acc: 0.6022
Epoch 6/31
135000/135000 [==============================] - 206s 2ms/sample - loss: 0.6921 - acc: 0.5681 - val_loss: 0.6921 - val_acc: 0.6208
Epoch 7/31
135000/135000 [==============================] - 194s 1ms/sample - loss: 0.6919 - acc:

In [48]:
#pickel_model=pickle.dump()
with open('./model_state_31.pkl','wb') as f:
    pickle.dump(model,f)

TypeError: can't pickle _thread.RLock objects

In [33]:
## for 0.5 drop out accuracy are for 
#1.epoch 25 71.64 40 99.73% 32 82%
##for 0.6 drop out accuracy are for
#1.epoch29 74.9% 31 83.43% 35  98.68%

# now our model is ready for the predictions.Let's test this model using test data

# read the test data

In [32]:
test=pd.read_csv("E:/TCS_data/test.csv")

In [33]:
test.head() #top 5 rown of the test data

,Id,Title,Body
0,6034196,Getting rid of site-specific hotkeys,<p>How do I disable site-specific hotkeys if (...
1,6034197,Nodes inside Cisco VPN. Incoming SSH requests ...,<p>I've a gateway-to-gateway VPN setup between...
2,6034198,Remove old vCenter servers from VMWare vSphere...,<p>After changing our vCenter servers recently...
3,6034199,Replace <span> element with var containing html,"<p>I have a variable i lifted the contents of,..."
4,6034200,Will PHP included html content affect my seo?,<p>Today i purchase a small CMS system. With t...


In [34]:
test.shape # shape of the test data

(10000, 3)

In [35]:
train=train.drop_duplicates() # drop_duplicates of test_data

In [36]:
for i in range(test.shape[0]):#remove html tags
    test.loc[i, 'Body'] = remove_html(test.loc[i,'Body'])

In [37]:
for column in test: # remove everything else which is scrap in the test dataset
    if(column=='Title' or column=='Body'):
        for i in range(test.shape[0]):
            test.loc[i, column] =clean_words(test.loc[i,column])

In [38]:
#tokenization
tokenized_doc_body_test=test['Body'].apply(lambda x:x.split())
tokenized_doc_title_test=test['Title'].apply(lambda x:x.split())
#remove stop_words
tokenized_doc_body_test=tokenized_doc_body_test.apply(lambda x:[item for item in x if item not in stop_words])
tokenized_doc_title_test=tokenized_doc_title_test.apply(lambda x:[item for item in x if item not in stop_words])
#de-tokenization
detokenized_doc_body_test=[]
detokenized_doc_title_test=[]
for i in range(test.shape[0]):
    t1=' '.join(tokenized_doc_body_test[i])
    t2=' '.join(tokenized_doc_title_test[i])
    detokenized_doc_body_test.append(t1)
    detokenized_doc_title_test.append(t2)
test['Body']= detokenized_doc_body_test
#test['Title']= detokenized_doc_title_test

In [41]:
test.head()

,Id,Title,Body
0,6034196,getting rid site specific hotkeys,disable site specific hotkeys already mapped o...
1,6034197,nodes inside cisco vpn incoming ssh requests a...,gateway gateway vpn setup linksys rv router ci...
2,6034198,remove old vcenter servers vmware vsphere clie...,changing vcenter servers recently old vcenter ...
3,6034199,replace span element var containing html,variable lifted contents div page like var ori...
4,6034200,php included html content affect seo,today purchase small cms system cms transform ...


In [43]:
test=shuffle(test,random_state=505)

In [44]:
test.head()

,Id,Title,Body
8781,6042977,ruby sequel last_insert_id method,insert need know id row
3348,6037544,good idea always define clustered index databa...,currently looking performance problem widely u...
5064,6039260,android api alphaanimation current alpha,try setup animation textview android api curre...
8057,6042253,jquery working object expected,following object var objectvar key key name it...
6115,6040311,dbus signals received window focused,trying add dbus support application seems ever...


In [39]:
test_body=test['Body'].values
#test_title=test['Title'].values#getting the values of the columns of body and title

In [45]:
#creating the term matrix for the test data
vocab_size=600
processor=TextPreprocessor(vocab_size)
processor.create_tokenizer(test_body)
body_text_test=processor.transform_text(test_body)
#body_title_test=processor.transform_text(test_tile)

In [ ]:
#total_data_test=np.hstack(body_title_test,body_text_test)#horizontal stacking up of test body and title columns

# finally we are at predictions step,the predicted_labels contains the tags for the test data.

In [46]:
predicted_labels=model.predict(body_text_test,batch_size=128)#predictions

In [47]:
predicted_labels

array([[0.06210372, 0.1209721 , 0.03556392, ..., 0.12605056, 0.06100351,
        0.06527048],
       [0.06656584, 0.12484285, 0.03850481, ..., 0.13428098, 0.06527838,
        0.06925729],
       [0.16988441, 0.23927364, 0.12391612, ..., 0.246604  , 0.1693069 ,
        0.17285177],
       ...,
       [0.03837588, 0.08661529, 0.01946965, ..., 0.09434745, 0.03767213,
        0.0413065 ],
       [0.21625131, 0.27989787, 0.17344266, ..., 0.28768718, 0.21595147,
        0.2212593 ],
       [0.1594089 , 0.22713488, 0.11529279, ..., 0.23425236, 0.15595236,
        0.1625748 ]], dtype=float32)

# this is my idea for the given problem statement using NLP and deep learning.